# Salt Lake City Network Visualization Example

This notebook demonstrates how to load and visualize the network data created by the main notebook.

In [ ]:
# Import required libraries
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import contextily as ctx

## Load the Network Data

Load the previously generated network data.

In [ ]:
# Load the network
try:
    tract_graph = nx.read_graphml('salt_lake_city_utah_usa_tract_network.graphml')
    print(f"Loaded network with {len(tract_graph.nodes())} nodes and {len(tract_graph.edges())} edges")
except Exception as e:
    print(f"Error loading network: {e}")
    print("Please run the main notebook first to generate the network data.")

In [ ]:
# Load node data with positions
try:
    nodes_df = pd.read_csv('salt_lake_city_utah_usa_tract_nodes.csv', index_col=0)
    
    # Extract position data
    pos = {}
    for idx, row in nodes_df.iterrows():
        # Check if pos column exists as tuple or as separate columns
        if 'pos' in row:
            pos[idx] = eval(row['pos']) 
        elif 'pos_x' in row and 'pos_y' in row:
            pos[idx] = (row['pos_x'], row['pos_y'])
            
    print(f"Loaded position data for {len(pos)} nodes")
except Exception as e:
    print(f"Error loading node data: {e}")
    print("Please run the main notebook first to generate the CSV files.")

## Visualize the Network

Create a simple visualization of the network.

In [ ]:
# Visualize the network
plt.figure(figsize=(12, 12))

# Convert position strings to tuples if needed
pos = {str(idx): eval(str(row['pos'])) if isinstance(row['pos'], str) else row['pos'] 
       for idx, row in nodes_df.iterrows()}

# Draw the network
nx.draw(
    tract_graph,
    pos=pos,
    node_size=50,
    node_color='blue',
    edge_color='red',
    alpha=0.7,
    with_labels=False
)

plt.title('Salt Lake City Census Tract Network')
plt.axis('off')
plt.show()

## Network Analysis

Perform some basic network analysis on the tract network.

In [ ]:
# Calculate basic network metrics
print("Network Metrics:")
print(f"Number of nodes: {len(tract_graph.nodes())}")
print(f"Number of edges: {len(tract_graph.edges())}")
print(f"Average degree: {sum(dict(tract_graph.degree()).values()) / len(tract_graph)}")

# Check if the graph is connected
is_connected = nx.is_connected(tract_graph)
print(f"Is connected: {is_connected}")

if not is_connected:
    # Get the number of connected components
    components = list(nx.connected_components(tract_graph))
    print(f"Number of connected components: {len(components)}")
    print(f"Sizes of connected components: {[len(c) for c in components]}")
else:
    # Calculate diameter and average path length
    diameter = nx.diameter(tract_graph)
    avg_path_length = nx.average_shortest_path_length(tract_graph)
    print(f"Network diameter: {diameter}")
    print(f"Average shortest path length: {avg_path_length:.2f}")